# Protein Ligand Complex

In this notebook we will demonstrate one way of setting up a simulation of a protein-ligand complex in OpenMM.

We will use the OpenMM package [openmmforcefields](https://github.com/openmm/openmmforcefields) and an external package called Open Force Field toolkit ([openff-toolkit](https://github.com/openforcefield/openff-toolkit)).

We will cover the following steps:

  - Loading in the ligand with `openff-toolkit`.
  - Parameterising the ligand force-field with `openmmforcefields`.
  - Combining the topologies.
  - Solvating and simulating.

*Note this notebook is based on the [openff-toolkit example](https://github.com/openforcefield/openff-toolkit/blob/stable/examples/toolkit_showcase/toolkit_showcase.ipynb) . We would like to give credit to the Open ForceField Authors.*



## Extra packages
<a id="packages1"></a>

We will need to install the additional python packages:

 - openmmforcefields
   - github: https://github.com/openmm/openmmforcefields
   - conda-forge: https://anaconda.org/conda-forge/openmmforcefields
 - openff-toolkit
   - github: https://github.com/openforcefield/openff-toolkit
   - conda-forge: https://anaconda.org/conda-forge/openff-toolkit

Both of these will be installed if you install openmmforcefields from conda-forge.


In [ ]:
!mamba install -y -c conda-forge openmmforcefields

## Imports

We need to be careful with the imports here because OpenMM and OpenFF have some objects with the same names. For this reason we no longer use the wildcard imports and adopt a more typical Python programming approach.

In [ ]:
# OpenMM imports
import openmm.app as app
import openmm as mm
import openmm.unit as unit

# OpenMM-forcefields imports
from openmmforcefields.generators import SMIRNOFFTemplateGenerator

# OpenFF-toolkit imports
from openff.toolkit import Molecule
from openff.toolkit import Topology as offTopology
from openff.units.openmm import to_openmm as offquantity_to_openmm

# other imports
from sys import stdout

## System

Our example system will be a complex of a benzene ligand and a lysozyme protein. The lysozyme is an antimicrobial protein that has been extensively studied by MD simulations. We can download the files from the github repo.

In [ ]:
# get the files
!wget https://raw.githubusercontent.com/openmm/openmm_workshop_july2023/main/section_1/benzene.sdf
!wget https://raw.githubusercontent.com/openmm/openmm_workshop_july2023/main/section_1/lysozyme.pdb

The benzene-lysozyme complex in shown in the figure below.

![benzene-lysozyme](./benzene_lysozyme.png)
**Figure.** Benzene-lysozyme complex.

Note that the files we are using have already been cleaned up (see [PDBFixer](https://github.com/openmm/pdbfixer) for more info). Additionally, the ligand is aligned with the protein and in an appropriate binding site. This is something you would need to do with a docking program before using OpenMM!

## Load in the molecules

The protein is a PDB file so we can load it as before with `PDBFile`. The benzene molecule is in SDF file format for which OpenMM does not have loaders. We need to use OpenFF toolkit to load it.

In [ ]:
protein_path = 'lysozyme.pdb'
ligand_path = 'benzene.sdf'

# Load a molecule from a SDF file
ligand = Molecule.from_file(ligand_path)

# Load in the protein from a PDB file
protein_pdb = app.PDBFile(protein_path)

### Creating the ForceField
<a id="createff"></a>

We now need to define the forcefield to use. For the protein we can use the standard forcefields already available in OpenMM. For the benzene molecule we will need to generate a forcefield template for it.

We can do this using the residue template generators for small molecules from the [openmmforcefields](https://github.com/openmm/openmmforcefields) package. There is a choice between the [Amber GAFF small molecule force field](http://ambermd.org/antechamber/gaff.html) or the [Open Force Field Initiative force fields](https://github.com/openforcefield/openff-forcefields).

For this example we will use [OpenFF SMIRNOFF](https://docs.openforcefield.org/projects/toolkit/en/stable/users/smirnoff.html).


In [ ]:
# Create the SMIRNOFF template generator with the default installed force field
smirnoff = SMIRNOFFTemplateGenerator(molecules=ligand)

# we can check which version of the force field is being used
print(smirnoff.smirnoff_filename)

# Create an OpenMM ForceField object with AMBER ff14SB and TIP3P
ff = app.ForceField('amber/protein.ff14SB.xml', 'amber/tip3p_standard.xml')

# add in the SMIRNOFF template generator
ff.registerTemplateGenerator(smirnoff.generator)

### Combine topologies and solvate
<a id="combine"></a>

We can convert from the OpenFF format topology to an OpenMM format topology and then use the OpenMM Modeller to combine the ligand and protein into a single topology. Once combined we can solvate as before.

In [ ]:
# make an OpenMM Modeller object with the protein
modeller = app.Modeller(protein_pdb.topology, protein_pdb.positions)

# make an OpenFF Topology of the ligand
ligand_off_topology = offTopology.from_molecules(molecules=[ligand])

# convert it to an OpenMM Topology
ligand_omm_topology = ligand_off_topology.to_openmm()

# get the positions of the ligand
ligand_positions = offquantity_to_openmm(ligand.conformers[0])

# add the ligand to the Modeller
modeller.add(ligand_omm_topology, ligand_positions)

# solvate
modeller.addSolvent(ff, padding=1.0*unit.nanometer, ionicStrength=0.15*unit.molar)


## Simulate

We can now simulate in the NVT ensemble. See the previous tutorial "Protein in water" for more explanation of this final step.

In [ ]:
system = ff.createSystem(modeller.topology, nonbondedMethod=app.PME, constraints=app.HBonds)
integrator = mm.LangevinMiddleIntegrator(300*unit.kelvin, 1/unit.picosecond, 0.002*unit.picoseconds)
simulation = app.Simulation(modeller.topology, system, integrator)

simulation.context.setPositions(modeller.positions)

simulation.minimizeEnergy(maxIterations=100)

simulation.context.setVelocitiesToTemperature(300*unit.kelvin)

simulation.reporters.append(app.PDBReporter('traj.pdb', 1000))

simulation.reporters.append(app.StateDataReporter(stdout, 100, step=True,
        potentialEnergy=True, temperature=True, speed=True))

print("Running simulation...")
simulation.step(1000)